In [1]:
import numpy as np
from coverage import *
from datetime import datetime, timezone, timedelta
import dataclasses
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import branca
import folium

from skyfield.framelib import itrs
from landsat import Instrument, Platform, Scene

! mkdir tmp

mkdir: tmp: File exists


In [2]:
start_dt = datetime.fromisoformat(Scene.start_utc)
num_days = 2

tles = gen_sats(
    # sat_nos=[Platform.norad_id] # How to best handle multiple platforms? (TLE vs. SPG4 model too)
    # sat_nos=[39084]
    sat_nos=[39084,49260]
)

inst = camera_model(
    name=Instrument.name, 
    fl=Instrument.focal_length_mm, 
    pitch=Instrument.pitch_um*1e-3, 
    h_pix=Instrument.rows, 
    v_pix=Instrument.cols, 
)

times = gen_times(
    start_yr=start_dt.year,
    start_mo=start_dt.month, 
    start_day=start_dt.day, 
    days=num_days, 
    step_min=Instrument.img_period)

xcell_size = ycell_size = .1

Satellite(s) Loaded from TLE:

[<EarthSatellite LANDSAT 8 catalog #39084 epoch 2022-06-15 18:38:09 UTC>]

[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2022-06-15 11:12:37 UTC>]

Propogation time: 
 2022-07-26 00:00:00+00:00 
to 
 2022-07-27 23:59:48+00:00

In [3]:
## Batch FOV generation over N satellites - TODO: build multiple sats into config/ main script
gdfs = []
for tle in tles:
    sat = tle[0]

    fov_df = forecast_fovs(sat, times, inst)

    xyz_dist_rates = sat.at(times).frame_xyz_and_velocity(itrs)
    fov_df['x_pos'], fov_df['y_pos'], fov_df['z_pos'] = xyz_dist_rates[0].km
    fov_df['x_vel'], fov_df['y_vel'], fov_df['z_vel'] = xyz_dist_rates[1].km_per_s

    gdfs.append(fov_df)

fov_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="EPSG:4326")

## Create cmap for unique satellites and create color column
sat_ids = list(fov_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)
fov_df['color'] = fov_df['id'].apply(cmap)

In [5]:
# fov_df.time = pd.to_datetime(fov_df.time)
fov_df["frac_days"] = fov_df.datetime.dt.dayofweek + fov_df.datetime.dt.hour/24 + fov_df.datetime.dt.minute/(24*60) + fov_df.datetime.dt.second/(24*60*60)
fov_df['time_gap'] = fov_df['frac_days'] - fov_df['frac_days'].shift(1)
# fov_df = fov_df.drop('datetime', axis=1)
fov_df

,datetime,satellite,id,time,geometry,x_pos,y_pos,z_pos,x_vel,y_vel,z_vel,color,frac_days,time_gap
0,2022-07-26 00:00:00+00:00,LANDSAT 8,139084,2022-07-26 00:00:00 UTC,"POLYGON ((48.69251 -82.41929, 51.23184 -80.795...",763.336641,749.121101,-7006.279929,6.705559,-3.480048,0.359449,#ff0000ff,1.000000,NaN
1,2022-07-26 00:00:22+00:00,LANDSAT 8,139084,2022-07-26 00:00:22 UTC,"POLYGON ((39.37295 -81.98888, 43.31485 -80.435...",910.518091,672.129420,-6996.479164,6.673917,-3.518846,0.531486,#ff0000ff,1.000255,0.000255
2,2022-07-26 00:00:44+00:00,LANDSAT 8,139084,2022-07-26 00:00:44 UTC,"POLYGON ((31.16828 -81.37681, 36.07937 -79.914...",1056.962644,594.306058,-6982.896512,6.638573,-3.555650,0.703240,#ff0000ff,1.000509,0.000255
3,2022-07-26 00:01:06+00:00,LANDSAT 8,139084,2022-07-26 00:01:06 UTC,"POLYGON ((24.14460 -80.61851, 29.62490 -79.255...",1202.589108,515.695254,-6965.539185,6.599550,-3.590427,0.874621,#ff0000ff,1.000764,0.000255
4,2022-07-26 00:01:28+00:00,LANDSAT 8,139084,2022-07-26 00:01:28 UTC,"POLYGON ((18.21617 -79.74632, 23.95641 -78.481...",1347.316865,436.341931,-6944.416412,6.556877,-3.623147,1.045537,#ff0000ff,1.001019,0.000255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15705,2022-07-27 23:58:20+00:00,LANDSAT 9,149260,2022-07-27 23:58:20 UTC,"POLYGON ((163.56990 46.82996, 161.26742 47.245...",-4675.803447,1509.042909,5094.035562,-4.450427,3.438390,-5.091799,#0000ffff,2.998843,0.000255
15706,2022-07-27 23:58:42+00:00,LANDSAT 9,149260,2022-07-27 23:58:42 UTC,"POLYGON ((163.05234 45.52560, 160.80283 45.934...",-4772.319345,1584.428542,4980.639227,-4.323363,3.414464,-5.216483,#0000ffff,2.999097,0.000255
15707,2022-07-27 23:59:04+00:00,LANDSAT 9,149260,2022-07-27 23:59:04 UTC,"POLYGON ((162.55460 44.21930, 160.35475 44.622...",-4866.014778,1659.262932,4864.530861,-4.194034,3.388278,-5.338335,#0000ffff,2.999352,0.000255
15708,2022-07-27 23:59:26+00:00,LANDSAT 9,149260,2022-07-27 23:59:26 UTC,"POLYGON ((162.07499 42.91118, 159.92174 43.308...",-4956.840767,1733.496440,4745.773504,-4.062518,3.359840,-5.457289,#0000ffff,2.999606,0.000255


In [6]:
# %%timeit
# get_inst_fov(sat, times[0], inst)

In [7]:
# fov_df.loc[fov_df.z_vel < 0]['asc_dsc'] = 'dsc'
conditions = [
    fov_df['z_vel'].lt(0),
    fov_df['z_vel'].gt(0)
]

choices = ['dsc','asc']
fov_df['asc_dsc'] = np.select(conditions, choices, default='undefined')

my_column_changes = fov_df["asc_dsc"].shift() != fov_df["asc_dsc"]
change = fov_df[my_column_changes]#[:100]
change['time_gap'] = change['frac_days'].shift(-1) - change['frac_days']

# plot_df = change[change.satellite=="LANDSAT 9"][1:]
# fig, ax = plt.subplots(1, figsize=(16,6))
# ax.barh(plot_df['asc_dsc'], plot_df.time_gap, left=plot_df.frac_days) #, color=df.color)

/Users/kevinlalli/miniconda3/envs/coverage/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
## Drop ascending pass FOVs
fov_df.loc[fov_df.asc_dsc=="asc", "geometry"] = None

## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world[world.continent != "Antarctica"]
# world = world[world.continent != "Seven seas (open ocean)"]

## Drop any FOVs not over land area
join = fov_df.sjoin(world, how="left").dropna()
fov_df.loc[join.index, 'mode'] = "SCIENCE"
fov_df.loc[~fov_df.index.isin(join.index), 'mode'] = "STANDBY"
fov_df.loc[fov_df['mode']=="STANDBY", "geometry"] = None

In [9]:
## Plotting FOVs

## Make a folium map
m = fov_df[fov_df['mode']=="SCIENCE"].drop('datetime', axis=1).explore(color="color", style_kwds={'fillOpacity':0.2}, tooltip=["satellite", "time"])

## Add WRS2
# wrs2 = gpd.read_file('./WRS2_descending_0/WRS2_descending.shp')
# wrs2 = wrs2.cx[xmin: xmax, ymin: ymax]
# folium.GeoJson(data=wrs2["geometry"], overlay=False).add_to(m)

## View or save
m#.save("./tmp/fovs_map.html")

In [10]:
## Coverage data analysis for single satellite/ batch of satellites

## Set AOI
# aoi =  world[world.name == "Brazil"].geometry
# aoi =  world[world.continent == "North America"].geometry
# aoi = world[world.name == "United States of America"].geometry # Includes Alaska...

## Or read in aoi from .geojson
aoi = gpd.read_file('./aois/eastern_us.geojson').geometry # ...so use AOI for subsection of US

## Filter fov_df by aoi
xmin, ymin, xmax, ymax= aoi.total_bounds
revisit_df = fov_df.cx[xmin: xmax, ymin: ymax]

## Create revisit map on regular grid
grid, grid_shape = calculate_revisits(revisit_df, aoi, grid_x=xcell_size, grid_y=ycell_size)
grid.n_visits.fillna(0).describe()
m = revisit_map(grid, grid_shape, grid_x=xcell_size, grid_y=ycell_size)
m#.save("./tmp/revisits_map.html")

In [11]:
# TODO: Fix saves to GeoJSON
import fiona

# grid.to_file('./tmp/all_revisits.geojson')

save_df = fov_df.loc[:,['satellite', 'geometry', 'time']].dropna()
save_df['satellite'] = save_df["satellite"].astype(str)
save_df['time'] = save_df["time"].astype(str)

## Save to geojson based on sat name
for satname in save_df.satellite.unique():
    with fiona.Env(OSR_WKT_FORMAT="WKT2_2018"):
        save_df[save_df.satellite==satname].to_file("./tmp/{}_fovs.geojson".format(satname.replace(" ", "_")), engine="pyogrio")

/Users/kevinlalli/miniconda3/envs/coverage/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/Users/kevinlalli/miniconda3/envs/coverage/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
